<a href="https://colab.research.google.com/github/mayanjabbaale/Facial-Emotion-Recognition/blob/main/Facial_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchinfo
import os
from google.colab import files
files.upload()


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
import kagglehub

path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fer2013' dataset.
Path to dataset files: /kaggle/input/fer2013


In [4]:
train_path = os.path.join(path, 'train')
test_path = os.path.join(path, 'test')

train_path, test_path

('/kaggle/input/fer2013/train', '/kaggle/input/fer2013/test')

In [10]:
from torchvision.datasets import ImageFolder
import PIL.Image as Image
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights

torch.manual_seed(54)
torch.cuda.manual_seed_all(54)

model = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
model_transforms = MobileNet_V3_Small_Weights.DEFAULT.transforms()

weights = MobileNet_V3_Small_Weights.DEFAULT
mean = model_transforms.mean
std = model_transforms.std

train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((96, 96)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

train_data = ImageFolder(train_path, transform=train_transforms)
test_data = ImageFolder(test_path, transform=test_transforms)

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)

imgs, labels = next(iter(train_dataloader))
class_names = train_data.classes


img, label = train_data[0]
img.shape, class_names[label]

(torch.Size([3, 96, 96]), 'angry')

In [ ]:
import matplotlib.pyplot as plt
import random
import torch

# Plot more images
# torch.manual_seed(42)
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.permute(2,1,0), cmap="gray")
    plt.title(class_names[label])
    plt.axis(False);

In [ ]:
from torchinfo import summary

for param in model.features.parameters():
    param.requires_grad = False

model.classifier[3] = nn.Linear(in_features=1024, out_features=7, bias=True)

model_summary = summary(model, input_size=[1, 3, 98, 98], col_names=['trainable'])
model

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from timeit import default_timer as timer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)

epochs = 20
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

def train_step(dataloader, model, loss_fn, optimizer, device):
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Forward pass
        y_logits = model(X)
        loss = loss_fn(y_logits, y)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        y_pred = torch.softmax(y_logits, dim=1).argmax(1)
        acc = accuracy_fn(y_true=y, y_pred=y_pred)
        train_acc += acc

    avg_loss = train_loss / len(dataloader)
    accuracy = train_acc / len(dataloader)
    return avg_loss, accuracy

def test_step(dataloader, model, loss_fn, device):
    model.eval() # Set model to evaluation mode
    test_loss, test_acc = 0, 0
    with torch.inference_mode(): # Disable gradient calculation for inference
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            y_logits = model(X)
            loss = loss_fn(y_logits, y)
            test_loss += loss.item()

            y_pred = torch.softmax(y_logits, dim=1).argmax(1)
            acc = accuracy_fn(y_true=y, y_pred=y_pred)
            test_acc += acc

    avg_loss = test_loss / len(dataloader)
    accuracy = test_acc / len(dataloader)
    return avg_loss, accuracy

start_time = timer()
for epoch in range(epochs):
    train_loss, train_acc = train_step(train_dataloader, model, loss_fn, optimizer, device)
    test_loss, test_acc = test_step(test_dataloader, model, loss_fn, device)

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%")

end_time = timer()
train_time = end_time - start_time

print("Training complete!")
if train_time < 60:
  print(f'Trained for {train_time:.3f} seconds')
else:
  train_time = train_time / 60.0
  print(f'Trained for {train_time:.3f} minutes')

torch.save(obj=model.state_dict(), f="facial_emotion_model.pth")

Using cuda device
Epoch 1/20 | Train Loss: 2.3460 | Train Acc: 28.93% | Test Loss: 1.6470 | Test Acc: 38.89%
Epoch 2/20 | Train Loss: 1.5444 | Train Acc: 42.30% | Test Loss: 1.4428 | Test Acc: 45.17%
Epoch 3/20 | Train Loss: 1.3767 | Train Acc: 48.07% | Test Loss: 1.3090 | Test Acc: 50.38%
Epoch 4/20 | Train Loss: 1.2713 | Train Acc: 51.73% | Test Loss: 1.2450 | Test Acc: 52.77%
Epoch 5/20 | Train Loss: 1.1990 | Train Acc: 54.70% | Test Loss: 1.1977 | Test Acc: 54.90%
Epoch 6/20 | Train Loss: 1.1372 | Train Acc: 57.01% | Test Loss: 1.1436 | Test Acc: 57.33%
Epoch 7/20 | Train Loss: 1.0845 | Train Acc: 59.35% | Test Loss: 1.1257 | Test Acc: 58.26%


In [ ]:
!ls

facial_emotion_model.pth  kaggle.json  sample_data


In [ ]:
model.eval()
fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    logits = model(img.unsqueeze(0))
    pred_class = class_names[torch.softmax(logits, 1).argmax(1).item()]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.permute(2,1,0), cmap="gray")
    plt.title(f'{class_names[label]} | {pred_class}')
    plt.axis(False);

In [ ]:
import gradio as gr

In [ ]:
def predict()
interface = gr.Interface()